# 1. Import and Install Dependencies

In [4]:
!pip install tensorflow==2.4.1 opencv-python mediapipe sklearn matplotlib

You should consider upgrading via the 'C:\Users\Parth Mehta\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import tensorflow as tf

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
   # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             #mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             #mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             #) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [7]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>


In [9]:
draw_landmarks(frame, results)

# 3. Extract Keypoint Values

In [10]:
len(results.left_hand_landmarks.landmark)

AttributeError: 'NoneType' object has no attribute 'landmark'

In [37]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [38]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [39]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() 
    if results.face_landmarks 
    else np.zeros(1404)


IndentationError: unexpected indent (<ipython-input-39-3fe2adb7da0b>, line 2)

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [8]:
result_test = extract_keypoints(results)

NameError: name 'results' is not defined

In [42]:
result_test

array([ 0.59639484,  0.41420001, -0.91864508, ...,  0.        ,
        0.        ,  0.        ])

In [43]:
np.save('0', result_test)

In [44]:
np.load('0.npy')

array([ 0.59639484,  0.41420001, -0.91864508, ...,  0.        ,
        0.        ,  0.        ])

# 4. Setup Folders for Collection

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join("C:/Users/Parth Mehta/OneDrive/Desktop/capstone/data") 

# Actions that we try to detect
actions = np.array(['Help','Hard of Hearing','Nice to Meet You','What is Up','What is your name',
                    'Please','Come here','I Understand','Which','Excuse Me'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [64]:
for action in actions: 
   for sequence in range(no_sequences):
     try:
       os.makedirs(os.path.join(DATA_PATH , action , str(sequence)))
     except:
       pass

# 5. Collect Keypoint Values for Training and Testing

In [65]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [66]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'Help': 0,
 'Hard of Hearing': 1,
 'Nice to Meet You': 2,
 'What is Up': 3,
 'What is your name': 4,
 'Please': 5,
 'Come here': 6,
 'I Understand': 7,
 'Which': 8,
 'Excuse Me': 9}

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
np.array(sequences).shape

(300, 30, 258)

In [14]:
np.array(labels).shape

(300,)

In [15]:
X = np.array(sequences)

In [16]:
X.shape

(300, 30, 258)

In [17]:
y = to_categorical(labels).astype(int)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [19]:
y_test.shape

(60, 10)

In [20]:
X_train.shape

(240, 30, 258)

In [21]:
y_train.shape

(240, 10)

# 7. Build and Train LSTM Neural Network

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Bidirectional, Dropout
from tensorflow.keras.callbacks import TensorBoard


In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [88]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [45]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [25]:
cp_best_val_loss = ModelCheckpoint(
      "SLD_val_loss", monitor='val_loss', mode = 'min', save_weights_only=True, save_best_only=True, verbose=1
)
cp_best_val_acc = ModelCheckpoint(
      "SLD_val_acc", monitor='val_categorical_accuracy', mode = 'max', save_weights_only=True, save_best_only=True, verbose=1
)

In [47]:
model.fit(X_train, y_train, epochs=200, callbacks=[cp_best_val_loss, cp_best_val_acc,tb_callback])

Epoch 1/200
8/8 [==============================] - 16s 867ms/step - loss: 2.7789 - categorical_accuracy: 0.4484
Epoch 2/200
3/8 [==========>...................] - ETA: 3s - loss: 1.9265 - categorical_accuracy: 0.3073

KeyboardInterrupt: 

In [27]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    '''
    Halts the training after reaching 60 percent accuracy

    Args:
      epoch (integer) - index of epoch (required but unused in the function definition below)
      logs (dict) - metric results from the training epoch
    '''

    # Check accuracy
    if(logs.get('accuracy') > 0.99):

      # Stop if threshold is met
      print("\naccuracy is higher than 0.6 so cancelling training!")
      self.model.stop_training = True

# Instantiate class
callback = myCallback()

In [26]:
dropout = 0.2
nodesizes = [256, 256, 128]


model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True),input_shape=(30, 258)))
model.add(Dropout(rate=dropout))  

for i in range(0,3):    
    model.add(Bidirectional(LSTM(nodesizes[i],return_sequences=True,activation='tanh')))
    model.add(Dropout(rate=dropout))  

model.add(Bidirectional(LSTM(256)))
model.add(Dropout(rate=dropout))

model.add(Dense(actions.shape[0],activation='softmax'))

model.compile(loss='categorical_crossentropy',
            optimizer='Adamax',
            metrics=['categorical_accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 30, 256)           396288    
_________________________________________________________________
dropout (Dropout)            (None, 30, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 30, 512)           1050624   
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 30, 512)           1574912   
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 512)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 30, 256)           6

In [56]:
history = model.fit(
    X_train, y_train, 
    epochs=100, 
    batch_size=30, 
    validation_data=(X_test,y_test), 
    shuffle=False,
    callbacks=[cp_best_val_loss, cp_best_val_acc,tb_callback])

Epoch 1/100
8/8 [==============================] - 18s 1s/step - loss: 2.2702 - categorical_accuracy: 0.1259 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - val_loss: 1.8566 - val_categorical_accuracy: 0.4000 - val_precision_3: 1.0000 - val_recall_3: 0.0167

Epoch 00001: val_loss improved from inf to 1.85662, saving model to SLD_val_loss

Epoch 00001: val_categorical_accuracy improved from -inf to 0.40000, saving model to SLD_val_acc
Epoch 2/100
8/8 [==============================] - 6s 710ms/step - loss: 1.7568 - categorical_accuracy: 0.4128 - precision_3: 0.6783 - recall_3: 0.0636 - val_loss: 1.5761 - val_categorical_accuracy: 0.3833 - val_precision_3: 0.5455 - val_recall_3: 0.1000

Epoch 00002: val_loss improved from 1.85662 to 1.57608, saving model to SLD_val_loss

Epoch 00002: val_categorical_accuracy did not improve from 0.40000
Epoch 3/100
8/8 [==============================] - 6s 713ms/step - loss: 1.3478 - categorical_accuracy: 0.5301 - precision_3: 0.8228 - recall_3: 0.219

8/8 [==============================] - 6s 737ms/step - loss: 0.0344 - categorical_accuracy: 0.9895 - precision_3: 0.9895 - recall_3: 0.9895 - val_loss: 0.3267 - val_categorical_accuracy: 0.9167 - val_precision_3: 0.9167 - val_recall_3: 0.9167

Epoch 00042: val_loss did not improve from 0.23992

Epoch 00042: val_categorical_accuracy did not improve from 0.95000
Epoch 43/100
8/8 [==============================] - 6s 764ms/step - loss: 0.0406 - categorical_accuracy: 0.9895 - precision_3: 0.9895 - recall_3: 0.9895 - val_loss: 0.3299 - val_categorical_accuracy: 0.9167 - val_precision_3: 0.9167 - val_recall_3: 0.9167

Epoch 00043: val_loss did not improve from 0.23992

Epoch 00043: val_categorical_accuracy did not improve from 0.95000
Epoch 44/100
8/8 [==============================] - 6s 758ms/step - loss: 0.0237 - categorical_accuracy: 0.9895 - precision_3: 0.9895 - recall_3: 0.9895 - val_loss: 0.3360 - val_categorical_accuracy: 0.9167 - val_precision_3: 0.9167 - val_recall_3: 0.9167

Epoc

KeyboardInterrupt: 

# 8. Make Predictions

In [29]:
res = model.predict(X_test)

In [30]:
actions[np.argmax(res[6])]

'Come here'

In [31]:
actions[np.argmax(y_test[6])]

'Come here'

# 9. Save Weights

In [60]:
model.save('lstmfinal.h5')

In [28]:
model.load_weights('lstmfinal.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [32]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [33]:
yhat = model.predict(X_test)

In [34]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [35]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[53,  0],
        [ 0,  7]],

       [[55,  0],
        [ 1,  4]],

       [[53,  1],
        [ 0,  6]],

       [[51,  0],
        [ 0,  9]],

       [[53,  0],
        [ 0,  7]],

       [[52,  0],
        [ 0,  8]],

       [[55,  0],
        [ 0,  5]],

       [[56,  0],
        [ 0,  4]],

       [[54,  0],
        [ 0,  6]],

       [[57,  0],
        [ 0,  3]]], dtype=int64)

In [36]:
accuracy_score(ytrue, yhat)

0.9833333333333333

# 11. Test in Real Time

In [37]:
from scipy import stats

In [38]:
colors = [(245,117,16), (117,245,16), (16,117,245),(128,0,0),(240,128,128),(255,215,0),(32,178,170),(72,209,204),(70,130,180),
          (25,25,112)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [58]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'image' is not defined

<Figure size 1296x1296 with 0 Axes>

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

What is your name
<class 'mediapipe.python.solution_base.SolutionOutputs'>
What is your name
<class 'mediapipe.python.solution_base.SolutionOutputs'>
What is your name
<class 'mediapipe.python.solution_base.SolutionOutputs'>
What is your name
<class 'mediapipe.python.solution_base.SolutionOutputs'>
What is your name
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Nice to Meet You
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.Sol

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.S

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutp

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.S

...

Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutput

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Excuse Me
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Excuse Me
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Excuse Me
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Excuse Me
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Excuse Me
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Excuse Me
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Excuse Me
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Nice to Meet You
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Nice to Meet You
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Nice to Meet You
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Nice to Meet You
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Nice to Meet You
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Nice to Meet You
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Nice to Meet You
<class 'media

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.S

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hard of Hearing
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I Understand
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I Understand
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I Understand
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I Understand
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I Understand
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I Understand
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I Understand

Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution